# Projeto 1 - Ciência dos Dados

Nome: Matheus Freitas Sant'Ana

Nome: Gabriel Salvator Benatar

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [735]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji
from emoji import UNICODE_EMOJI

In [736]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\matfs\Desktop\Ciência dos Dados\P2\P1-Classificador-Naive-Bayes


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [737]:
dados = pd.read_excel('Uber.xlsx')
dados['Classe'] = dados.Classe.astype('category')

In [738]:
dados['Classe'].cat.categories = ['Irrelevante', 'Relevante']

In [739]:
dados.tail(10)

Treinamento       Classe
290  aí que preguiça desse uber machista/xenofóbico...    Relevante
291        eu preciso muito parar de andar de uber mds  Irrelevante
292  @catarinalegal pegar uns uber mais estranhos t...  Irrelevante
293  @moisesstocco @wilsondenotari @valdecirdelecl1...  Irrelevante
294  vontade de comer um lanche e nem uma lanchonet...  Irrelevante
295  a pronto o uber ta tocando uma sofrência braba...  Irrelevante
296       @_diana_mota_ acho que vais ser o meu uber😂👌  Irrelevante
297  muito bom esse novo recurso da uber do motoris...    Relevante
298             mood: conversando em inglês com o uber    Relevante
299  mo burocracia p eu conseguir sair da escola de...  Irrelevante

In [740]:
dados.Classe.value_counts()

Irrelevante    212
Relevante       88
Name: Classe, dtype: int64

Há muito mais tweets irrelevantes do que relevantes. Vamos tentar deixar a quantidade mais ou menos parecida.

In [741]:
dados_irr = dados.loc[dados.Classe == 'Irrelevante', :]
dados_irr = dados_irr.loc[:130, :]

In [742]:
dados_irr.Classe.value_counts()

Irrelevante    88
Relevante       0
Name: Classe, dtype: int64

In [743]:
dados_r = dados.loc[dados.Classe == 'Relevante', :]

In [744]:
dados_r.Classe.value_counts()

Relevante      88
Irrelevante     0
Name: Classe, dtype: int64

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Função que limpa a pontuação e outros símbolos irrelevantes para a classificação dos _tweets_.

In [745]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@_]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
    

Limpando os _tweets_ do _DataFrame_

In [746]:
for i in range(len(dados['Treinamento'])):
    dados['Treinamento'][i] = cleanup(dados['Treinamento'][i].lower())

C:\Users\matfs\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [747]:
dados.head()

Treinamento       Classe
0  pior coisa é ouvir de gente privilegiada que a...  Irrelevante
1  eu morrendo de calor e o uber geladinho tudo q...  Irrelevante
2  to lembrando aqui do dia em que a mairine e a ...  Irrelevante
3   lucasp711 to cortando o cabelo  se ainda tive...  Irrelevante
4  rt  jungsbloomie  acabei o dinheiro da semana ...  Irrelevante

Criando os _DataFrames_ das categorias de classificação.

In [748]:
dados_irr = dados.loc[dados.Classe == 'Irrelevante', :]
dados_irr = dados_irr.loc[:130, :]

In [749]:
dados_r = dados.loc[dados.Classe == 'Relevante', :]

Aqui corrigimos o index de cada _DataFrame_ (eles tinham herdado o index do _DataFrame_ total).

In [750]:
for i in range(len(dados_r.index.values)):
    dados_r.index.values[i] = i 

In [751]:
for i in range(len(dados_irr.index.values)):
    dados_irr.index.values[i] = i 

Criando uma lista com as palavras de cada _tweet_ do _DataFrame_ dos irrelevantes.

In [752]:
todas_palavras_irrelevante = []
for i in range(len(dados_irr.Treinamento)):
    todas_palavras_irrelevante.append(dados_irr['Treinamento'][i].split())
todas_palavras_irrelevante[:2]

[['pior',
  'coisa',
  'é',
  'ouvir',
  'de',
  'gente',
  'privilegiada',
  'que',
  'anda',
  'de',
  'carro',
  'todo',
  'dia',
  'seja',
  'uber',
  'ou',
  'carro',
  'do',
  'pai',
  'que',
  'vai',
  'ter',
  'que',
  'andar',
  'de',
  'ônibus',
  'realmente',
  'essas',
  'pessoas',
  'precisam',
  'amadurecer'],
 ['eu',
  'morrendo',
  'de',
  'calor',
  'e',
  'o',
  'uber',
  'geladinho',
  'tudo',
  'q',
  'eu',
  'queria']]

Essa lista é composta de várias outras listas, em que cada uma delas corresponde às palavras de certo _tweet_. Vamos fazer com que a lista de palavras seja apenas UMA grande lista composta apenas pelas _strings_.

In [753]:
todas_palavras_irr = []
for i in range(len(todas_palavras_irrelevante)):
    for j in range(len(todas_palavras_irrelevante[i])):
        todas_palavras_irr.append(todas_palavras_irrelevante[i][j])
todas_palavras_irr[:35]

['pior',
 'coisa',
 'é',
 'ouvir',
 'de',
 'gente',
 'privilegiada',
 'que',
 'anda',
 'de',
 'carro',
 'todo',
 'dia',
 'seja',
 'uber',
 'ou',
 'carro',
 'do',
 'pai',
 'que',
 'vai',
 'ter',
 'que',
 'andar',
 'de',
 'ônibus',
 'realmente',
 'essas',
 'pessoas',
 'precisam',
 'amadurecer',
 'eu',
 'morrendo',
 'de',
 'calor']

Vamos tornar essa lista em um _DataFrame_.

In [754]:
serie_Irr = pd.Series(todas_palavras_irr)
serie_Irr.head()

0     pior
1    coisa
2        é
3    ouvir
4       de
dtype: object

Todo o processo feito acima é repetido para o _DataFrame_ dos _tweets_ relevantes.

In [755]:
todas_palavras_relevante = []
for i in range(len(dados_r.Treinamento)):
    todas_palavras_relevante.append(dados_r['Treinamento'][i].split())

In [756]:
todas_palavras_r = []
for i in range(len(todas_palavras_relevante)):
    for j in range(len(todas_palavras_relevante[i])):
        todas_palavras_r.append(todas_palavras_relevante[i][j])
todas_palavras_r[:5]

['rt', 'leandra', 'bdz', 'meninas', 'um']

In [757]:
serie_r = pd.Series(todas_palavras_r)
serie_r.head()

0         rt
1    leandra
2        bdz
3    meninas
4         um
dtype: object

Fazendo as tabelas com as contagens absolutas.

In [758]:
tabela_r = serie_r.value_counts()
tabela_r.head()

uber    89
de      61
e       51
que     42
o       41
dtype: int64

In [759]:
tabela_irr = serie_Irr.value_counts()
tabela_irr.head()

uber    89
o       52
de      44
e       42
que     36
dtype: int64

Fazendo as tabelas com as contagens relativas

In [760]:
tabela_relativa_r = (serie_r.value_counts(True)*100).round(2)
tabela_relativa_r.head()

uber    4.98
de      3.42
e       2.86
que     2.35
o       2.30
dtype: float64

In [761]:
tabela_relativa_irr = (serie_Irr.value_counts(True)*100).round(2)
tabela_relativa_irr.head()

uber    5.15
o       3.01
de      2.55
e       2.43
que     2.08
dtype: float64

Fazendo todo o processo acima só que para o _DataFrame_ total.

In [762]:
todas_palavras_v1 = []
for i in range(len(dados.index.values)):
    todas_palavras_v1.append(dados.Treinamento[i].split())

In [763]:
todas_palavras = []
for i in range(len(todas_palavras_v1)):
    for j in range(len(todas_palavras_v1[i])):
        todas_palavras.append(todas_palavras_v1[i][j])
todas_palavras[:5]

['pior', 'coisa', 'é', 'ouvir', 'de']

In [764]:
serie_todas = pd.Series(todas_palavras)
serie_todas.head()

0     pior
1    coisa
2        é
3    ouvir
4       de
dtype: object

In [765]:
tabela_todas = serie_todas.value_counts()
tabela_todas.head()

uber    295
de      166
o       159
e       145
que     107
dtype: int64

In [766]:
tabela_relativa_todas = (serie_todas.value_counts(True)*100).round(2)
tabela_relativa_todas.head()

uber    5.11
de      2.87
o       2.75
e       2.51
que     1.85
dtype: float64

Função do classificador Naive Bayes

In [767]:
def naive_bayes(frase, tabela_relativa_r, tabela_relativa_irr):
    
    #Filtrando a frase tirando pontuação e símbolos irrelevantes
    frase = cleanup(frase.lower())
    
    #Separando as palavras da frase e colocando-as em uma lista.
    lista_palavras = frase.split()
    
    #Quando há o uso de emojis e eles estão 'colados', separá-los para que sejam
    #considerados palavras diferentes.
    k = 0
    for m in range(len(lista_palavras)):
        for i in lista_palavras[m]:
            if i in UNICODE_EMOJI:
                k += 1
        if k == len(lista_palavras[m]):
            sub = ",".join(lista_palavras[m])
            a = sub.split(",")
            lista_palavras.remove(lista_palavras[m])
            for s in range(len(a)):
                lista_palavras.append(a[s])
        k = 0
        
    #Contando o total de palavras em cada categoria com as contagens absolutas.
    totalR = tabela_r.value_counts().sum()
    totalIrr = tabela_irr.value_counts().sum()
    total = totalR + totalIrr
    
    #P(R)
    probR = totalR/total
    
    #P(Rc)
    probIrr=totalIrr/total
    
    #P(frase|R)
    probFraseDadoR = 100
    for i in lista_palavras:
        if i in tabela_relativa_r:
            probFraseDadoR *= tabela_relativa_r['{}'.format(i)]
        else:
            probFraseDadoR *= 0.01
    
    #P(frase|Rc)
    probFraseDadoIrr = 100
    for i in lista_palavras:
        if i in tabela_relativa_irr:
             probFraseDadoIrr *= tabela_relativa_irr['{}'.format(i)]
        else:
            probFraseDadoIrr *= 0.01
    
    #P(R|frase)
    probRDadoFrase = probFraseDadoR*probR
    
    #P(Rc|frase)
    probIrrDadoFrase = probFraseDadoIrr*probIrr
    
    if probRDadoFrase > probIrrDadoFrase:
        return('Relevante, {}'.format(probRDadoFrase))
    else:
        return('Irrelevante, {}'.format(probIrrDadoFrase))
    

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [768]:
naive_bayes('meu deus o uber que eu peguei hoje era filho de italiano 😭😷😷😷', tabela_relativa_r, tabela_relativa_irr)

'Relevante, 3.721395848035707e-10'

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**